### Get results from Wandb

In [9]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [10]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,"{'act_f1': 0.35194659127169736, 'rew_loss': 0....","{'env': {'game': 'Assault', 'type': 'atari', '...",2zmcrr6s
1,"{'_step': 28000, 'act_f1': 0.1314340947832686,...","{'env': {'game': 'Alien', 'type': 'atari', 'fr...",1vcuejmv
2,"{'idm_loss': 2.3104390292167665, 'reward_f1': ...","{'env': {'game': 'Amidar', 'type': 'atari', 'f...",2x5eqtig
3,"{'rew_loss': 0.017572961137630044, 'best_metri...","{'env': {'game': 'Asterix', 'type': 'atari', '...",33lkzzf9
4,"{'max_grad_norm': 0.8564444112181664, 'best_me...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",195cpev4
...,...,...,...
1192,"{'_wandb': {'runtime': 20220}, '_runtime': 201...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1193,"{'_timestamp': 1659280381, 'negative_sim': 0.5...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1194,"{'_runtime': 18375, '_timestamp': 1659280529, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1195,"{'pos_neg_diff': 0.810436166524887, 'positive_...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [11]:
group_name = 'baseline'
exp_name = 'trajformer_demon_cons'
model_path = '0/10/model.pth'

In [12]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [13]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_16205/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
177,"{'_step': 234370, 'act_acc': 0.777087814924193...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",qw5ydgw7,UpNDown
178,"{'rew_loss': 0, 'loss': 0.9963398272105994, 'e...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",2p2dv198,Seaquest
187,"{'loss': 1.1132206543674317, 'idm_loss': 2.889...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",3i95d2a2,PrivateEye
188,"{'rew_loss': 0, 'pos_neg_diff': 0.482187891717...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",2vtlezvg,Qbert
189,"{'idm_loss': 2.9605588615748295, 'obs_loss': 0...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",3qdi7gm4,RoadRunner
190,"{'neg_sim': 0.42287328378940986, '_runtime': 5...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",3bmpzkip,Pong
203,"{'act_acc': 0.6577479771482266, 'idm_acc': 0.0...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",tl35e3x8,Krull
204,"{'reward_ratio': 0.04745237261863093, 'max_gra...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",3c8278yl,KungFuMaster
205,"{'act_f1': 0.22211560024668167, 'reward_f1': 0...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",3txh36kj,MsPacman
206,"{'act_f1': 0.25682677007931665, 'idm_acc': 0.0...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",289ul4r7,Kangaroo


### 1. Restore Saved Models

In [ ]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown qw5ydgw7


### Save to artifact

In [ ]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

### Remove artifact

In [ ]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)